In [0]:
# Data Science Libraries
import pandas as pd
import numpy as np

# Data Processing
import itertools
import functools
from functools import lru_cache

# Output to Excel
from shutil import copyfile

# Visualization
from IPython.display import display, HTML, Markdown

Main User Interface: https://ftgdev.service-now.com/x/fof/loss-triangles-2/loss-triangles-page

In [0]:
@lru_cache(maxsize=None)
def load_data(
    CLAIMS_LOCATION: str = "",
) -> pd.DataFrame:
    """
    Load insurance data from the provided file paths.

    This function attempts to load dataframes from the provided paths in the order:
    Parquet, CSV, and then Excel. If successful in one format, it stops further attempts.

    Parameters:
    - CLAIMS_LOCATION (str): Path to the claims data file.

    Returns:
    pd.DataFrame: The Claims dataframe.

    Raises:
    - ValueError: If the files are not in one of the expected formats.

    Examples:
    >>> premiums, claims = load_data("premiums.parquet", "claims.parquet")
    Dataset was in PARQUET format.
    """
    formats = [
        ("parquet", pd.read_parquet),
        ("csv", pd.read_csv),
        ("excel", pd.read_excel),
    ]

    for file_format, loader in formats:
        try:
            claims_df = loader(CLAIMS_LOCATION)
            print(f"Dataset was in {file_format.upper()} format.")
            return claims_df
        except Exception as e:
            print(
                f"Failed to load data in {file_format.upper()} format due to: {str(e)}"
            )

    raise ValueError(
        "Claims file is not in an accepted format of: Parquet, CSV, or Excel."
    )

In [0]:
def get_static_data(
    BOOK: str,
) -> tuple[pd.DataFrame, str, str, list[str],]:
    if BOOK == "King":
        CLAIMS_LOCATION = "/dbfs/FileStore/ID/kmh_book_10_04_23.xlsx"
        FULL_CLAIMS_DF = load_data(CLAIMS_LOCATION).rename(
            columns={
                "OS_loss": "OS_Loss",
            }
        )
        X_AXIS_DATE_FIELD = "Accounting_Date"
        CLAIM_NUMBER_COLUMN = "claim_nbr"
        LOSS_COLUMNS = ["Paid_Loss", "Paid_ALE", "OS_Loss", "OS_ALE"]

    elif BOOK == "Park":
        CLAIMS_LOCATION = "/dbfs/FileStore/ID/kp_book_10_05_23.xlsx"
        FULL_CLAIMS_DF = load_data(CLAIMS_LOCATION).rename(
            columns={
                "OS_loss": "OS_Loss",
            }
        )
        FULL_CLAIMS_DF = FULL_CLAIMS_DF.loc[FULL_CLAIMS_DF["ASL"] == 5.1]

        X_AXIS_DATE_FIELD = "Accounting_Date"
        CLAIM_NUMBER_COLUMN = "claim_nbr"
        LOSS_COLUMNS = ["Paid_Loss", "Paid_ALE", "OS_Loss", "OS_ALE"]

    elif BOOK == "AVT":
        CLAIMS_LOCATION = "/dbfs/FileStore/ID/avt_book_10_05_23.xlsx"
        FULL_CLAIMS_DF = load_data(CLAIMS_LOCATION).rename(
            columns={
                "acc_date": "Acc_Date",
                "policy_eff_date": "Policy_Date",
                "Reporting_date": "Report_Date",
                "accounting_date": "Accounting_Date",
                "claim_number": "claim_nbr",
                "paid_loss": "Paid_Loss",
                "paid_ale": "Paid_ALE",
                "os_loss": "OS_Loss",
                "os_ale": "OS_ALE",
                "salvage": "Salvage",
                "subrogation": "Subrogation",
            }
        )
        FULL_CLAIMS_DF = FULL_CLAIMS_DF.loc[FULL_CLAIMS_DF["NISS_ASL"] == 51.0]

        X_AXIS_DATE_FIELD = "Accounting_Date"  # accounting date
        CLAIM_NUMBER_COLUMN = "claim_nbr"
        LOSS_COLUMNS = ["Paid_Loss", "Paid_ALE", "OS_Loss", "OS_ALE"]

    return (
        FULL_CLAIMS_DF,
        X_AXIS_DATE_FIELD,
        CLAIM_NUMBER_COLUMN,
        LOSS_COLUMNS,
    )

In [0]:
def get_time_interval_fields(
    df: pd.DataFrame,
    X_AXIS_DATE_FIELD: str,
    Y_AXIS_DATE_FIELD: str,
    TIME_INTERVAL: str = "quarterly",
) -> tuple[pd.DataFrame, dict]:
    df[X_AXIS_DATE_FIELD] = pd.to_datetime(df[X_AXIS_DATE_FIELD])
    df[Y_AXIS_DATE_FIELD] = pd.to_datetime(df[Y_AXIS_DATE_FIELD])

    time_interval_axis_mapping = {}

    if TIME_INTERVAL == "quarterly":
        x_axis_name = "Months_Developed_AQ"
        y_axis_name = "First_AQ"
        df[x_axis_name] = df[X_AXIS_DATE_FIELD].dt.to_period("Q").astype(str)
        df[y_axis_name] = df[Y_AXIS_DATE_FIELD].dt.to_period("Q").astype(str)
        time_interval_axis_mapping["triangle_column_name"] = "AQ"

    elif TIME_INTERVAL == "monthly":
        x_axis_name = "Months_Developed"
        y_axis_name = "First_Month"
        df[x_axis_name] = (
            df[X_AXIS_DATE_FIELD].dt.year.astype(str)
            + "-"
            + df[X_AXIS_DATE_FIELD].dt.month.astype(str)
        )
        df[y_axis_name] = (
            df[Y_AXIS_DATE_FIELD].dt.year.astype(str)
            + "-"
            + df[Y_AXIS_DATE_FIELD].dt.month.astype(str)
        )
        time_interval_axis_mapping["triangle_column_name"] = "AM"

    elif TIME_INTERVAL == "yearly":
        x_axis_name = "Years_Developed"
        y_axis_name = "First_Year"
        df[x_axis_name] = df[X_AXIS_DATE_FIELD].dt.year.astype(str)
        df[y_axis_name] = df[Y_AXIS_DATE_FIELD].dt.year.astype(str)
        time_interval_axis_mapping["triangle_column_name"] = "AY"

    else:
        assert "TIME_INTERVAL parameter must be in ['quarterly', 'monthly', 'yearly']"

    time_interval_axis_mapping["x-axis"] = x_axis_name
    time_interval_axis_mapping["y-axis"] = y_axis_name

    return (df, time_interval_axis_mapping)

In [0]:
def get_cumulative_metrics(
    df: pd.DataFrame,
    CLAIM_NUMBER_COLUMN: str = "claim_nbr",
    LOSS_COLUMNS: list[str] = ["Paid_Loss", "Paid_ALE", "OS_Loss", "OS_ALE"],
    time_interval_axis_mapping: dict = {},
    X_AXIS_DATE_FIELD: str = "",
) -> pd.DataFrame:
    assert (
        time_interval_axis_mapping != {}
    ), "time_interval_axis_mapping must not be an empty dict"

    df = df.sort_values(
        [
            CLAIM_NUMBER_COLUMN,
            time_interval_axis_mapping["y-axis"],
            time_interval_axis_mapping["x-axis"],
            X_AXIS_DATE_FIELD,
        ]
    )
    df[LOSS_COLUMNS] = df.groupby([CLAIM_NUMBER_COLUMN])[LOSS_COLUMNS].cumsum()
    return df

In [0]:
def get_last_values_during_developed_period(
    df: pd.DataFrame,
    CLAIM_NUMBER_COLUMN: str = "claim_nbr",
    LOSS_COLUMNS: list[str] = ["Paid_Loss", "Paid_ALE", "OS_Loss", "OS_ALE"],
    time_interval_axis_mapping: dict = {},
) -> pd.DataFrame:
    assert (
        time_interval_axis_mapping != {}
    ), "time_interval_axis_mapping must not be an empty dict"

    agg_dict = {col: "last" for col in LOSS_COLUMNS}
    df = (
        df.groupby(
            [
                CLAIM_NUMBER_COLUMN,
                time_interval_axis_mapping["y-axis"],
                time_interval_axis_mapping["x-axis"],
            ]
        )
        .agg(agg_dict)
        .reset_index(drop=False)
    )
    return df

In [0]:
def get_period_claim_pivot(
    df: pd.DataFrame,
    CLAIM_NUMBER_COLUMN: str,
    time_interval_axis_mapping: dict = {},
    TIME_INTERVAL: str = "quarterly",
) -> tuple[pd.DataFrame, str, str]:
    assert (
        time_interval_axis_mapping != {}
    ), "time_interval_axis_mapping must not be an empty dict"

    def _get_period_list(
        start_period: str, end_period: str, TIME_INTERVAL: str = "quarterly"
    ):
        freq_time_interval_mapping = {
            "quarterly": "Q",
            "monthly": "M",
            "yearly": "Y",
        }
        start_period = pd.Period(
            start_period, freq=freq_time_interval_mapping[TIME_INTERVAL]
        )
        end_period = pd.Period(
            end_period, freq=freq_time_interval_mapping[TIME_INTERVAL]
        )

        period_range = pd.period_range(
            start=start_period,
            end=end_period,
            freq=freq_time_interval_mapping[TIME_INTERVAL],
        )

        period_list = period_range.astype(str).tolist()

        return period_list

    start_period, end_period = (
        min(
            df[time_interval_axis_mapping["x-axis"]].min(),
            df[time_interval_axis_mapping["y-axis"]].min(),
        ),
        max(
            df[time_interval_axis_mapping["x-axis"]].max(),
            df[time_interval_axis_mapping["y-axis"]].max(),
        ),
    )

    period_list = _get_period_list(start_period, end_period, TIME_INTERVAL)
    claim_nbr_list = df[CLAIM_NUMBER_COLUMN].unique().astype(str).tolist()

    period_claim_nbr_list = list(itertools.product(period_list, claim_nbr_list))

    df_period_claim_pivot = pd.DataFrame(
        period_claim_nbr_list,
        columns=[time_interval_axis_mapping["x-axis"], CLAIM_NUMBER_COLUMN],
    )

    df_quarter_claim_pivot = df_period_claim_pivot.sort_values(
        [CLAIM_NUMBER_COLUMN, time_interval_axis_mapping["x-axis"]]
    )[[CLAIM_NUMBER_COLUMN, time_interval_axis_mapping["x-axis"]]]

    return (df_quarter_claim_pivot, start_period, end_period)

In [0]:
def merge_with_period_claim_pivot(
    df: pd.DataFrame,
    df_period_claim_pivot: pd.DataFrame,
    CLAIM_NUMBER_COLUMN: str,
    time_interval_axis_mapping: dict = {},
) -> pd.DataFrame:
    assert (
        time_interval_axis_mapping != {}
    ), "time_interval_axis_mapping must not be an empty dict"

    claims_triangle_df = df.merge(
        df_period_claim_pivot,
        on=[CLAIM_NUMBER_COLUMN, time_interval_axis_mapping["x-axis"]],
        how="outer",
    )

    claims_triangle_df[time_interval_axis_mapping["y-axis"]] = claims_triangle_df.groupby(
        [CLAIM_NUMBER_COLUMN]
    )[time_interval_axis_mapping["y-axis"]].ffill()

    claims_triangle_df[
        [
            "Paid_Loss",
            "Paid_ALE",
            "OS_Loss",
            "OS_ALE",
            "Total_Paid",
            "Total_OS",
            "Total_Incurred",
            "Total_ALE",
            "Total_Loss",
        ]
    ] = (
        claims_triangle_df.sort_values(
            [
                CLAIM_NUMBER_COLUMN,
                time_interval_axis_mapping["y-axis"],
                time_interval_axis_mapping["x-axis"],
            ]
        )
        .groupby(CLAIM_NUMBER_COLUMN)[
            [
                "Paid_Loss",
                "Paid_ALE",
                "OS_Loss",
                "OS_ALE",
                "Total_Paid",
                "Total_OS",
                "Total_Incurred",
                "Total_ALE",
                "Total_Loss",
            ]
        ]
        .ffill()
    )

    return claims_triangle_df

In [0]:
def get_loss_triangle_on_target_metric(
    df: pd.DataFrame,
    start_quarter: str,
    end_quarter: str,
    TARGET_LOSS_METRIC: str = "Total_Paid",
    TIME_INTERVAL: str = "quarterly",
    time_interval_axis_mapping: dict = {},
) -> tuple[list[list[float]], int, list[str]]:
    assert (
        time_interval_axis_mapping != {}
    ), "time_interval_axis_mapping must not be an empty dict"

    def _get_period_list(start_period: str, end_period: str, TIME_INTERVAL: str = "quarterly"):
        freq_time_interval_mapping = {
            "quarterly": "Q",
            "monthly": "M",
            "yearly": "Y",
        }
        start_period = pd.Period(start_period, freq=freq_time_interval_mapping[TIME_INTERVAL])
        end_period = pd.Period(end_period, freq=freq_time_interval_mapping[TIME_INTERVAL])

        period_range = pd.period_range(
            start=start_period, end=end_period, freq=freq_time_interval_mapping[TIME_INTERVAL]
        )

        period_list = period_range.astype(str).tolist()

        return period_list
    
    def _get_period_data(y_axis_period: str) -> list[float]:
        return [
            df.loc[
                (df[time_interval_axis_mapping["y-axis"]] == y_axis_period)
                & (df[time_interval_axis_mapping["x-axis"]] == x_axis_period)
            ][TARGET_LOSS_METRIC].sum()
            for x_axis_period in _get_period_list(
                y_axis_period, end_quarter, TIME_INTERVAL
            )
        ]

    y_axis_list = _get_period_list(start_quarter, end_quarter, TIME_INTERVAL)
    triangle_list = [_get_period_data(y_period) for y_period in y_axis_list]

    max_len = max(len(row) for row in triangle_list)
    triangle_list = [
        row + [np.nan] * (max_len - len(row)) for row in triangle_list
    ]

    return triangle_list, max_len, y_axis_list

In [0]:
def get_claims_triangle(
    triangle_list: list[list[float]],
    max_len: int,
    y_axis_list: list[str],
    TIME_INTERVAL: str = "quarterly",
    time_interval_axis_mapping: dict = {},
) -> pd.DataFrame:
    time_interval_to_iteration_mapping = {"quarterly": 3, "monthly": 1, "yearly": 12}
    development_month_list = (
        np.arange(
            time_interval_to_iteration_mapping[TIME_INTERVAL],
            (max_len + 1) * time_interval_to_iteration_mapping[TIME_INTERVAL],
            time_interval_to_iteration_mapping[TIME_INTERVAL],
        )
        .astype(int)
        .astype(str)
        .tolist()
    )
    dataframe_dtype_dict = {
        development_month: float for development_month in development_month_list
    }
    dataframe_dtype_dict[time_interval_axis_mapping["triangle_column_name"]] = str

    claims_triangle = pd.DataFrame(
        np.hstack((np.array(y_axis_list).reshape(-1, 1), np.array(triangle_list))),
        columns=[time_interval_axis_mapping["triangle_column_name"]]
        + development_month_list,
    ).astype(dataframe_dtype_dict)
    return claims_triangle

In [0]:
def get_ldf_from_claims_triangle(
    claims_triangle: pd.DataFrame, TIME_INTERVAL: str = "quarterly"
) -> pd.DataFrame:
    """
    Calculate the loss development factor from the claims triangle.

    Parameters:
    - triangle_ldf_df (pd.DataFrame): The triangular formatted dataframe.

    Returns:
    - pd.DataFrame: The dataframe updated with loss development factors.
    """
    time_interval_to_iteration_mapping = {"quarterly": 3, "monthly": 1, "yearly": 12}

    triangle_ldf_df = claims_triangle.copy()
    columns = triangle_ldf_df.columns[1:]

    for i in range(len(columns) - 1):
        col_name = columns[i]
        next_col_name = columns[i + 1]

        triangle_ldf_df[col_name] = 1 / np.where(
            triangle_ldf_df[next_col_name] == 0,
            np.nan,
            triangle_ldf_df[col_name].div(triangle_ldf_df[next_col_name]),
        )
    triangle_ldf_df[columns[-1]] = np.nan

    age_to_age_columns = [
        "{}-{}".format(
            columns[i], int(columns[i]) + time_interval_to_iteration_mapping[TIME_INTERVAL]
        )
        for i in range(len(columns))
    ]
    triangle_ldf_df.columns = [triangle_ldf_df.columns[0]] + age_to_age_columns

    triangle_ldf_df.replace([np.inf, -np.inf], np.nan, inplace=True)

    return triangle_ldf_df

In [0]:
def get_loss_triangles_from_parameters(
    df: pd.DataFrame,
    X_AXIS_DATE_FIELD: str,
    Y_AXIS_DATE_FIELD: str,
    CLAIM_NUMBER_COLUMN: str,
    LOSS_COLUMNS: list[str],
    TARGET_LOSS_METRIC: str,
    TIME_INTERVAL: str,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    def _update_insurance_totals(df: pd.DataFrame) -> pd.DataFrame:
        df["Total_Paid"] = df["Paid_Loss"] + df["Paid_ALE"]
        df["Total_OS"] = df["OS_Loss"] + df["OS_ALE"]
        df["Total_Incurred"] = df["Total_Paid"] + df["Total_OS"]
        df["Total_ALE"] = df["Paid_ALE"] + df["OS_ALE"]
        df["Total_Loss"] = df["Paid_Loss"] + df["OS_Loss"]
        return df

    (claims_df, time_interval_axis_mapping) = get_time_interval_fields(
        df, X_AXIS_DATE_FIELD, Y_AXIS_DATE_FIELD, TIME_INTERVAL
    )
    claims_df = get_cumulative_metrics(
        claims_df,
        CLAIM_NUMBER_COLUMN,
        LOSS_COLUMNS,
        time_interval_axis_mapping,
        X_AXIS_DATE_FIELD,
    )
    claims_df = get_last_values_during_developed_period(
        claims_df, CLAIM_NUMBER_COLUMN, LOSS_COLUMNS, time_interval_axis_mapping
    )

    claims_df = _update_insurance_totals(claims_df)

    (df_quarter_claim_pivot, start_quarter, end_quarter) = get_period_claim_pivot(
        claims_df, CLAIM_NUMBER_COLUMN, time_interval_axis_mapping, TIME_INTERVAL
    )

    claims_triangle_df = merge_with_period_claim_pivot(
        claims_df,
        df_quarter_claim_pivot,
        CLAIM_NUMBER_COLUMN,
        time_interval_axis_mapping,
    )

    (triangle_list, max_len, First_AQ_list) = get_loss_triangle_on_target_metric(
        claims_triangle_df,
        start_quarter,
        end_quarter,
        TARGET_LOSS_METRIC,
        TIME_INTERVAL,
        time_interval_axis_mapping,
    )

    claims_triangle = get_claims_triangle(
        triangle_list, max_len, First_AQ_list, TIME_INTERVAL, time_interval_axis_mapping
    )

    claims_triangle_ldf = get_ldf_from_claims_triangle(claims_triangle, TIME_INTERVAL)

    return (claims_triangle, claims_triangle_ldf)

In [0]:
def get_ldf_weighted_average_and_ftu(
    df: pd.DataFrame,
    df2: pd.DataFrame,
    TIME_INTERVAL: str,
) -> pd.DataFrame:
    def _get_ldf_average(df: pd.DataFrame, TIME_INTERVAL: str) -> np.ndarray:
        if TIME_INTERVAL == "yearly":
            # df.mean() treats the "AY" column as a aggregation value
            ldf_average = df.mean().values[1:]
        else:
            ldf_average = df.mean().values
        return ldf_average.reshape(1, -1)

    def _get_weighted_ldf_average(df: pd.DataFrame, df2: pd.DataFrame) -> np.ndarray:
        # Get dataframe into numpy format and drop first column (TIME_INTERVAL column)
        claim_amounts = df.iloc[:, 1:].to_numpy()

        # Set last non-NaN value to NaN
        last_non_nan_indices = np.argmax(np.isnan(claim_amounts), axis=1) - 1
        last_non_nan_indices[last_non_nan_indices == -1] = claim_amounts.shape[1] - 1
        claim_amounts[np.arange(claim_amounts.shape[0]), last_non_nan_indices] = np.nan

        # Set NaN values to 0.0 for weighting
        claim_amounts[np.isnan(claim_amounts)] = 0.0

        # Get dataframe into numpy format and drop first column (TIME_INTERVAL column)
        age_to_age_factors = df2.iloc[:, 1:].to_numpy()

        # Get the weights
        columnwise_summation = np.sum(claim_amounts, axis=0)
        weights = claim_amounts / columnwise_summation[np.newaxis, :]

        # Calculate the weights age-to-age factors
        weighted_age_to_age_factors = weights * age_to_age_factors

        # Calculate the summed weighted age-to-age factors
        summed_weighted_age_to_age_factors = np.nansum(
            weighted_age_to_age_factors, axis=0
        ).reshape(1, -1)

        return summed_weighted_age_to_age_factors

    ldf_average = _get_ldf_average(df2, TIME_INTERVAL)[:, :-1]
    weighted_ldf_average = _get_weighted_ldf_average(df, df2)[:, :-1]

    def get_ftu_cell_value(weighted_ldf_average: np.ndarray) -> float:
        ftu_cell = np.cumprod(weighted_ldf_average, axis=1)[0, -1]
        return ftu_cell

    ftu = np.array(
        [
            get_ftu_cell_value(weighted_ldf_average[:, i:])
            for i in np.arange(0, len(weighted_ldf_average[0]))
        ]
    ).reshape(1, -1)

    all_ldf_caption_values = np.hstack(
        (
            np.array([["Average", "Weighted Average", "FTU"]]).T,
            np.vstack((ldf_average, weighted_ldf_average, ftu)),
        )
    )

    ldf_averages_df = pd.DataFrame(
        all_ldf_caption_values,
        columns=["Metric"] + df2.columns[1:-1].tolist(),
    )
    ldf_averages_df[df2.columns[1:-1].tolist()] = ldf_averages_df[
        df2.columns[1:-1].tolist()
    ].astype(float)
    return ldf_averages_df

In [0]:
def get_pandas_ldf_styling(df: pd.DataFrame) -> pd.DataFrame:
    def highlight_values(val):
        if val > 10:
            return "background-color: red"
        elif val > 2.0:
            return "background-color: orange"
        elif val > 1.00:
            return "background-color: yellow"
        elif val > 0.95:
            return "background-color: lime"
        elif val > 0.05:
            return "background-color: blue"
        else:
            return "background-color: none"

    styled_df = df.style.applymap(highlight_values, subset=df.columns[1:])

    return styled_df

In [0]:
def run(
    USE_STATIC_DATA: bool = True,
    Y_AXIS_DATE_FIELDS: list[str] = [
        "Acc_Date",
        # "Policy_Date",
        # "Report_Date",
        # "Treaty_Date",
    ],
    TARGET_LOSS_METRICS: list[str] = [
        "Total_Paid",
        "Total_Incurred",
        "Total_ALE",
        "Total_Loss",
    ],
    TIME_INTERVALS: list[str] = ["monthly", "quarterly", "yearly"],
    LOCAL_FILEPATH: str = "/tmp/ldf_output_permutation_test_tempfile.xlsx",
    OUTPUT_DBFS_FILEPATH: str = "/dbfs/mnt/datascience/ldf_output_permutation_test.xlsx",
) -> list[tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]]:
    if USE_STATIC_DATA:
        BOOK = "King"
        (
            FULL_CLAIMS_DF,
            X_AXIS_DATE_FIELD,
            CLAIM_NUMBER_COLUMN,
            LOSS_COLUMNS,
        ) = get_static_data(BOOK)

    triangle_outputs = []
    with pd.ExcelWriter(LOCAL_FILEPATH, engine="openpyxl") as writer:
        for idx, combination in enumerate(itertools.product(
            Y_AXIS_DATE_FIELDS, TARGET_LOSS_METRICS, TIME_INTERVALS
        )):
            (Y_AXIS_DATE_FIELD, TARGET_LOSS_METRIC, TIME_INTERVAL) = combination

            (claims_triangle, claims_triangle_ldf) = get_loss_triangles_from_parameters(
                FULL_CLAIMS_DF,
                X_AXIS_DATE_FIELD,
                Y_AXIS_DATE_FIELD,
                CLAIM_NUMBER_COLUMN,
                LOSS_COLUMNS,
                TARGET_LOSS_METRIC,
                TIME_INTERVAL,
            )

            ldf_averages_df = get_ldf_weighted_average_and_ftu(
                claims_triangle, claims_triangle_ldf, TIME_INTERVAL
            )

            triangle_outputs.append(
                (claims_triangle, claims_triangle_ldf, ldf_averages_df)
            )

            combined_df = pd.concat(
                [claims_triangle, claims_triangle_ldf, ldf_averages_df],
                axis=1,
                keys=["claims_triangle", "claims_triangle_ldf", "ldf_averages_df"],
            )

            sheet_name = f"comb_{idx}"
            combined_df.to_excel(writer, sheet_name=sheet_name)

    copyfile(LOCAL_FILEPATH, OUTPUT_DBFS_FILEPATH)

    return triangle_outputs

In [0]:
triangle_outputs = run()